# Homework 8 Solutions

## FINM 36700 - 2024

### UChicago Financial Mathematics
* Tobias
* tobiasdelpozo@uchicago.edu

### HBS Case: *Long-Term Capital Management*

Note: these solutions are very similar to last year's, given that LTCM imploded in 1998.

## 1.1 



<font color='red'>

Securities traded:

LTCM tried to trade on market mispricing and arbitrage, Relative Value and Convergence trades. They go long-short on these arbitrages. Use leverage to trade bigger principal on these small mispricings and try to hedge out their positions via their long-short trades. They primarily used derivatives, in the form of swaps to achieve these positions.

LTCM was also heavily involved income and credit, and they also have sizeable positions in equities. In all these asset classes, they trade a large number of securities, across global markets.

Trading frequency:

LTCM's trading frequency varied according to their strategies. Their largest investment in the form of convergence trades had a long term trading horizon and frequency (weeks or months). They are not trying to arbitrage intraday movements and nor do they make long-term directional bets. 

Skewness: 

They are picking up pennies in front of a bulldozer. So, many small wins. They seek small positive returns using leverage and do not bet significantly on any specific events. Have lower skewness than SPY. However, they are susceptible to extreme market events (it was the Russian currency crisis that brought them down).

Forecasing: 

Build models to find mispricing and the reason behind the mispricing. Then forecast their P&L on these trades. Their forecast is not better because of better mathematical model (the convergence trade/ relative value theory is not the edge), it is their knowledge of the market.
</font>

## 1.2

<font color='red'>

1. Efficient financing: Their edge was on financing and funding, along with their proprietary trading and modelling capabilities.
2. Fund Size: They had a larger AUM, meaning they could lever at favorable rates
3. Collatralization: Better collateralize these positions. (pay lower haircuts)
4. Long-term Horizon: Long term commitment of capital from investors as well as availability of credit line
5. Liquidity and Hedging: LTCM has in place many mechanisms to ensure liquidity. They also avoid taking too much default risk or explicit directional bets. 

</font>

## 1.3

<font color='red'>


Collateral haircuts:

The haircuts go up in a market disruption event leading to unfavorable collateral terms for LTCM in terms of funding a spread trade. For most trades, LTCM obtains 100% financing on a fully collateralized basis. Furthermore, LTCM stress tests the haircuts across its asset classes.

Repo maturity:

In an adverse situation, where their credit risk goes up, they wont be able to secure these longer term repos which were favorable to their trades. LTCM goes against the norm by entering into relatively long-maturity repo. While much of it is overnight, LTCM uses contracts that typically have maturity of 6-12 months. Furthermore, LTCM manages their aggregate repo maturity.

Equity redemption:

If in a convergence trade, the two securities, before converging, diverge a lot, LTCM are facing redemption risk from their investors at a time where the Margin calls need them to furhter finance their strategies. Equity Redemption at a unfavorable time also leads LTCM to unwind their positions at unfavorable rates leading to further losses of capital. The firm is highly levered, so equity funding risk is especially important. LTCM restricts redemptions of equity year by year. The restriction is particularly strong in that unredeemed money is re-locked.

Loan access:

Loan access can be tough to come by in times of a crisis, leading to a further decline in the fund's performance. For debt funding, LTCM negotiated a revolving loan that has no Material Adverse Change clause. Thus, the availability of debt funding is not so highly correlated with fund performance. 

</font>

## 1.4

<font color='red'>

LTCM required counterparties to maitain the collateral balance via a 'two-way mark to market process on a daily basis. Thus the cash flow coming in from the counterparties mark to market would fund LTCM's outflow for the mark to market call on their offsetting position.

LTCM als also estimated theoretical worst case haircuts it would face in adverse market situations. Forecasting these worst case liquidity LTCM was able to better structure its financing so as not to liquidate its positions solely due to these adverse market events.

LTCM attempts to account for liquidity risk quantitatively by adjusting security correlations. For short-term horizons, LTCM assumes positive correlation between all trade cat- egories. Even if their net exposure to a strategy flips sides, they still assume positive correlation to the new net position

</font>

## 1.5

<font color='red'>
Currently since there were no extreme market events, leverage risk is not a concern, but still a potential threat for LTCM. Given the size of their commited capital and fewer opportunites for the excess capital to enhance LTCM's return, they are considering returning some of the investments made, which would reduce the leverage.

Note: the amount of "true" leverage is also frequently misreported. The reason being that SEC filings require the reporting of the gross notional exposure, not the net exposure! As an example, consider [this article](https://www.cnn.com/2023/08/15/investing/michael-burry-stock-market-crash/index.html). It claims that Michael Burry "bet" $1.6 billion on a market crash. In reality, his exposure is $1.6 billion; he achieved this by buying put options for much, much, cheaper (potentially as low as ~$10m in premiums; capping his losses at $10m).
</font>

## 1.6

<font color='red'>

About a year after the time of the case, the fund loses most of its value due to non-converging trades. So clearly there is some risk!

Positions are subject to liquidity risk. If market liquidity dries up or the markets become segmented, the divergent spreads can persist for a long time. This indeed happens later to LTCM. The trades that get them in trouble ultimately pay off, but not before LTCM blew up. LTCM believed it can exit these convergence trades if they become too unprofitable. However, a stop-loss order is not the same as a put option. If the price jumps discontinuously through the stop-loss, then it is ineffective.

Or a market may be paralyzed/illiquid when trying to execute the stop-loss. A put option does not need to worry about price impact, whereas a stop-loss does. Finally, a stop-loss ensures that an investor sells as soon as a security price hits a worst-case scenario, ensuring unfavorable market timing.

</font>

## 2.1 a,b

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import sys

sys.path.append("../cmds/")
import TA_utils as ta


plt.style.use("ggplot")

# Read ltcm_exhibits_data.xlsx and get sheet Exhibit 2
ltcm = pd.read_excel(
    "../data/ltcm_exhibits_data.xlsx",
    sheet_name="Exhibit 2",
    skiprows=2,
    index_col=0,
    parse_dates=[0],
).iloc[:-4, :]
ltcm.index.name = "Date"

# Get SPY returns from returns (total) in gmo_analysis_data.xlsx
spy = pd.read_excel(
    "../data/gmo_analysis_data.xlsx",
    sheet_name="total returns",
    index_col=0,
    parse_dates=[0],
)[["SPY"]]
us3m = (
    pd.read_excel(
        "../data/gmo_analysis_data.xlsx",
        sheet_name="risk-free rate",
        index_col=0,
        parse_dates=[0],
    )[["TBill 3M"]]
    / 12
)

# Make index a dateindex not a timestamp for ltcm
ltcm.index = pd.to_datetime(
    ltcm.index, format="%Y-%m-%d"
) + pd.tseries.offsets.MonthEnd(0)

ltcm = ltcm[["Net Monthly Performanceb", "Gross Monthly Performancea"]].dropna()
ltcm = ltcm.rename(
    columns={
        "Net Monthly Performanceb": "LTCM Net",
        "Gross Monthly Performancea": "LTCM Gross",
    }
)

# Join on month and year of the index
df = ltcm.join(spy, how="inner").join(us3m, how="inner")

# Convert to EXCESS returns
df.loc[:, ["LTCM Net", "LTCM Gross", "SPY"]] = df.loc[
    :, ["LTCM Net", "LTCM Gross", "SPY"]
].subtract(df["TBill 3M"], axis=0)

ta.calc_performance_metrics(df[["LTCM Net", "LTCM Gross", "SPY"]]).T

,LTCM Net,LTCM Gross,SPY
Annualized Return,0.192996,0.287974,0.175565
Annualized Volatility,0.108234,0.133308,0.106577
Annualized Sharpe Ratio,1.783142,2.160224,1.647307
Annualized Sortino Ratio,2.018056,2.723823,3.067598
Skewness,-0.879601,-0.313006,-0.436963
Excess Kurtosis,3.888872,1.998281,-0.198906
VaR (0.05),-0.017566,-0.019766,-0.044823
CVaR (0.05),-0.051983,-0.05665,-0.048045
Min,-0.105133,-0.105133,-0.050021
Max,0.080267,0.112267,0.07498


## 2.1 c

<font color='red'>

Comparing the Net Monthly Performance of LTCM and Excess returns of SPY, LTCM displays a higher return, with a very similar volatility to SPY. Thus the sharpe ratio of LTCM net of fee and other charges is slightly higher than SPY's.

The excess net returns of LTCM however, underperform SPY with similar volatility levels and thus have a slightly lower sharpe ratio.

However, looking at other moments, LTCM Net returns are more negatively skewed and have a significantly fatter tail compared to SPY, indicating the presence of heavy negative monthly returns over the sample period. Although, since the VaR of LTCM is lower compared to SPY, the indication is that these negative returns are fewer in frequency.

</font>

## 2.2 a

In [2]:
ta.calc_iterative_regression(
    df[["LTCM Net", "LTCM Gross"]], df[["SPY"]], one_to_many=True
).T

,LTCM Net,LTCM Gross
Alpha,0.178842,0.266126
Beta,0.080618,0.124449
Downside Beta,-0.212736,-0.161578
R-Squared,0.006302,0.009899
Treynor Ratio,2.393954,2.314005
Information Ratio,1.657604,2.006282
Tracking Error,0.008991,0.011054


## 2.2 b

<font color='red'>

Definitely not. Low $\beta$ and low $R^2$, as well as high alpha mean that the fund is not tracking the market and is in no way a closet indexer. This is likely because they are trading fixed income primarily, and most of their trades are pairs/arbitrage trades, meaning that by definition they don't have exposure to the market.

Additionally, these "convergence" trades are *always* long-short, meaning that they are neutral to most factors.
</font>

## 2.2 c

<font color='red'>

Very high net alpha at 13% per year, so it delivers high excess returns beyond market exposure.

</font>

## 2.3 a

In [3]:
df["SPY_Squared"] = df["SPY"] ** 2
df["SPY_Put"] = np.maximum(-0.03 - df["SPY"], 0)
df["SPY_Call"] = np.maximum(df["SPY"] - 0.03, 0)

ta.calc_iterative_regression(
    df[["LTCM Net", "LTCM Gross"]], df[["SPY", "SPY_Squared"]], one_to_many=True
).T

,LTCM Net,LTCM Gross
Alpha,0.208732,0.300593
R-Squared,0.016729,0.019039
SPY Beta,0.125942,0.176714
SPY_Squared Beta,-2.765492,-3.189034
Information Ratio,1.944869,2.276660
Tracking Error,0.107324,0.132033


## 2.3 b,c,d

<font color='red'>

b.

The quadratic factor imprroves the $R^2$ marginally, this is likely because we're just adding more features to the regression, so $R^2$ must increase. So it doesn't help much.

The huge negative beta on SPY squared is a feature of the factor. The monthly returns are small, thus the squared returns are even smaller, thus the beta has to be larger in magnitude to fit these small returns properly.

c.

Since the beta to SPY squared returns is negative, LTCM's market exposure behaves as if it were short the market options. The beta to SPY can be interpreted as the delta of the market option and the beta to SPY squared as the gamma to market options. Since the gamma of an option is always positive, LTCM seems to be short the positive gamma or short the market options.

d.


For a big monthly return, the negative beta for SPY Squared would lead to heavy underperformance of LTCM returns. Big market movements would lead to big underperformance of LTCM. Thus, LTCM seems to be taking on a negative exposure to market volatility underperforming big deviations in the market and the performance not being impacted much by small deviations.

</font>

## 2.4 a

In [4]:
ta.calc_iterative_regression(
    df[["LTCM Net", "LTCM Gross"]], df[["SPY", "SPY_Put", "SPY_Call"]], one_to_many=True
).T

,LTCM Net,LTCM Gross
Alpha,0.161220,0.237455
R-Squared,0.032718,0.042924
SPY Beta,0.339976,0.488780
SPY_Put Beta,0.937522,1.407126
SPY_Call Beta,-0.716244,-0.968403
Information Ratio,1.514535,1.820761
Tracking Error,0.106448,0.130415


## 2.4 b,c,d

<font color='red'>

b. 

Long puts short calls. This position is called a [risk reversal](https://www.investopedia.com/terms/r/riskreversal.asp).

c.

We are more long puts than we are short calls, based on the magnitude of the betas. So the put-like behavior dominates.

d. 

We know that they are short volatility, but we **cannot conclude this** based on this regression alone, and, in reality, this regression actually implies the opposite. Namely, being long options is being long vol. So, we are long a lot more put options than call options, meaning that in a vacuum, we are long vol. However, this depends on the *strikes* of the options. So if the call-like factor is closer to ATM than the put-like factor, then we might actually be short vol, since the calls will have a higher vega than the puts. However, if the strikes are symmetric (equally OTM), then we are long vol.

But; we can say that we are long downside vol, and short upside vol.

</font>

## 3.0 (Data Processing)

In [5]:
risk_free_rates = pd.read_excel(
    "../data/fx_rf_data.xlsx",
    sheet_name="risk-free rates",
    index_col=0,
    parse_dates=[0],
)
exchange_rates = pd.read_excel(
    "../data/fx_rf_data.xlsx", sheet_name="exchange rates", index_col=0, parse_dates=[0]
)

risk_free_rates = risk_free_rates / 252
risk_free_rates = np.log(1 + risk_free_rates)
exchange_rates = np.log(exchange_rates)

## 3.1

In [6]:
# Get the log return relative to USD, defined as
# ret = spot_{t+1} - spot_{t} + rf_{foreign,t,t+1} - rf_{USD,t,t+1}

rets = pd.DataFrame(data=0, index=exchange_rates.index, columns=exchange_rates.columns)
for col in exchange_rates.columns:
    rets[col] = (
        exchange_rates[col].shift(-1) # Gets spot_{t+1}
        - exchange_rates[col]         # Gets spot_{t}
        + risk_free_rates[col]        # Risk free rate at time t -> t+1
        - risk_free_rates["USD"]      # Local risk free rate at time t -> t+1
    )

rets = rets.dropna()

ta.calc_performance_metrics(rets, adj=252)[
    ["Annualized Return", "Annualized Volatility", "Annualized Sharpe Ratio"]
].T

,JPY,EUR,GBP,MXN,CHF
Annualized Return,-0.029160,-0.023121,-0.024334,0.013094,0.003116
Annualized Volatility,0.099118,0.090045,0.095666,0.129978,0.106857
Annualized Sharpe Ratio,-0.294196,-0.256769,-0.254368,0.100743,0.029159


<font color='red'>

JPY, EUR, and GBP all have negative returns. MXN and CHF have had positive returns. Something interesting is the volatilities are all very similar, meaning that the primary difference in sharpe is due to mean return.

</font>

## 3.2 a,b,c

<font color='red'>

a.

All of them do. UIP states that the mean return of these currencies positions should be zero as the change in the spot fx rate is completely explained by the changes in risk free rates. However, none of the mean returns for the currencies are 0.

b.

Over the sample, the MXN has had the highest sharpe, at 0.1.

c. 

All currencies except CHF and MXN earned a negative or near zero excess returns in USD terms. JPY performed the worst (see TA review 8 for more on JPY).

</font>


## 3.3 a

In [7]:
fx_hldg_reg = {"Alpha": [], "Beta": [], "R-Squared": []}

for col in exchange_rates.columns:
    # y = fx_{t+1} - fx_{t}
    y = exchange_rates.shift(-1)[col] - exchange_rates[col]

    # X = rf_{USD, t, t+1} - rf_{foreign, t, t+1}
    X = risk_free_rates["USD"] - risk_free_rates[col]

    summ = ta.calc_univariate_regression(y, X, intercept=True, adj=252)
    fx_hldg_reg["Alpha"].append(summ.loc[col, "Alpha"])
    fx_hldg_reg["Beta"].append(summ.loc[col, "Beta"])
    fx_hldg_reg["R-Squared"].append(summ.loc[col, "R-Squared"])

fx_hldg_reg_summary = pd.DataFrame(fx_hldg_reg, index=exchange_rates.columns).T
fx_hldg_reg_summary

,JPY,EUR,GBP,MXN,CHF
Alpha,-0.021877,-0.031513,-0.025228,-0.073561,0.003703
Beta,0.351755,2.160192,6.810272,-0.808766,0.906945
R-Squared,0.000013,0.000298,0.001537,0.000020,0.000050


## 3.3 b

<font color='red'>

Note that since $s_{t+1} - s_t$ indicated depreciation of USD if this number is positive, and appreciation of USD if this is negative.

i.

If foreign risk-free rate increases relative to US, then that means our X is now negative.

Thus, for the USD to appreciate, we would need a positive regression beta (since negative * positive = negative = USD appreciation). 

From the regression, we see JPY, EUR, and GBP having a positive beta and thus would have lower exchange rates in case the risk-free rate of Japan increases. Thus USD would relatively strengthen against JPY, GBP, EUR.

ii.

If risk-free rate of a currency were to increase relative to the US rate, the currencies with a negative beta in the previous regression would see an increase in the fx rates (USD per foreign currency). This indicates that there will be a relative weakening of the USD as a dollar would now buy less of those currencies.

MXN and CHF both have a negative beta to USD. Thus, USD would experience a relative weakening relative to these 2 currencies.

iii.

Indicated by the R-Squared in the regression, the FX predictibility seems to be strongest in case of GBP. However, it should be noted that this R-Squared is still fairly low and might not indicate towards a strong enough prediction.

</font>

## 3.4

In [8]:
fx_prem_df = pd.DataFrame(
    data=0, index=exchange_rates.index, columns=exchange_rates.columns
)

for col in exchange_rates.columns:
    # Get expected value as :
    # alpha + (beta - 1) * (rf_{USD, t, t+1} - rf_{foreign, t, t+1})
    expected_value = (1 / 252) * fx_hldg_reg_summary.loc[
        "Alpha", col
    ] + (fx_hldg_reg_summary.loc["Beta", col] - 1) * (
        risk_free_rates["USD"] - risk_free_rates[col]
    )
    fx_prem_df[col] = expected_value

fx_prem_df = fx_prem_df.dropna()

# Calculate how often each is > 0
(
    ((fx_prem_df > 0).sum() / len(fx_prem_df)).to_frame("Predicted > 0") * 100
).style.format("{:.2f}%")

,Predicted > 0
JPY,0.00%
EUR,3.78%
GBP,24.88%
MXN,76.08%
CHF,96.49%


## 3.4 b,c

<font color='red'>
b. 

CHF displays the highest consistency in producing positive FX risk premium followed by MXN, and GBP. On the other hand JPY and EUR has a negative FX risk premium during (nearly) all months in the sample.

a.

Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1.

Since from 3.4.a JPY returns are not 0, we could flip the carry trade to short the JPY and go long the USD. This would give us a positive risk premia from the carry trade. Similarly, we could do this for EUR and GBP as well.

</font>